In [ ]:
!pip install transformers groq torch


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
import pandas as pd
from groq import Groq

ModuleNotFoundError: No module named 'groq'

In [ ]:

client = Groq(api_key="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")

def promo (link):
    prompt = (
            "You are a cybersecurity expert specializing in cyber threat intelligence. Given the link below, please "
            "generate a maximum of 5 multiple-choice questions with four possible options each.\n"
            "Follow these requirements:\n"
            "DO NOT INCLUDE \"Here are the 5 multiple-choice questions based on the provided link:\"at the start of the answer or anything like it"
            "0. Do not include any introductory text or additional commentary outside the specified format."
            "1. Question Format: Each question must have four options. The options should be challenging and require "
            "careful consideration. Avoid creating options that could be interpreted as correct under different circumstances.\n"
            "2. Target Audience: The questions should be suitable for security professionals with three to five years of "
            "experience in cyber threat intelligence. Avoid generic questions such as 'What is the objective?', 'Which "
            "operating system can be targeted?'.\n"
            "3. Content Coverage: Aim to cover various sections of the document to ensure a comprehensive evaluation of "
            "the candidate’s knowledge. Include context-specific questions that require an understanding of the document’s "
            "content.\n"
            "4. Technical Precision: Use precise terminology and concepts relevant to cyber threat intelligence. Incorporate "
            "situational or scenario-based questions where applicable.\n"
            "5. Include Technique IDs and Names: Ensure that all questions, where applicable, mention both the ID and the "
            "full name of the MITRE ATT&CK pattern technique.\n"
            "6. Premise Inclusion: Each question should include a premise indicating it pertains to MITRE ATT&CK, "
            "specifying the relevant platform (Enterprise, ICS, or Mobile) where necessary.\n"
            """7. Output Format: Return the output in plain text format(must be separated by ;).
            Question, Option A, Option B, Option C, Option D, Correct Answer (A, B, C, D) in 1 line for each quest do not include question number
            The format should be as follows:
               [Your question here];[First option];[Second option];[Third option];[Fourth option];[Correct answer letter]\n
               Do not include any introductory text or additional commentary outside the specified format.\n"""
            "8. Important: Do not include any sentences like 'Here are the questions' or similar. Only provide the formatted questions and answers.\n"
            f"link :\n {link}"
        )
    response = client.chat.completions.create(model='llama3-70b-8192', messages=[
      {
        'role': 'user',
        'content':prompt,
      },
    ])
    #print(response['message']['content'])
    return response.choices[0].message.content.strip()


In [ ]:
def sheetparser(file):
  df = pd.read_excel(file, sheet_name='techniques')
  for idx, row in df.iterrows():
      link = row["url"]
      output = promo(link)
      print(f"current file :{file} index : {idx} ")
      rows = output.strip().split('\n')
      # Split each row by the semicolon delimiter
      split_rows = [row.split(';') for row in rows]
      # Create a DataFrame
      df = pd.DataFrame(split_rows)
      # Save the DataFrame to a TSV file
      df.to_csv('outputraw.tsv', sep='\t',mode='a', index=False, header=False)


In [ ]:
sheetparser('enterprise-attack-v15.1.xlsx')
sheetparser('ics-attack-v15.1.xlsx')
sheetparser('mobile-attack-v15.1.xlsx')

current file :enterprise-attack-v15.1.xlsx index : 0 
current file :enterprise-attack-v15.1.xlsx index : 1 
current file :enterprise-attack-v15.1.xlsx index : 2 
current file :enterprise-attack-v15.1.xlsx index : 3 
current file :enterprise-attack-v15.1.xlsx index : 4 
current file :enterprise-attack-v15.1.xlsx index : 5 
current file :enterprise-attack-v15.1.xlsx index : 6 
current file :enterprise-attack-v15.1.xlsx index : 7 
current file :enterprise-attack-v15.1.xlsx index : 8 
current file :enterprise-attack-v15.1.xlsx index : 9 
current file :enterprise-attack-v15.1.xlsx index : 10 
current file :enterprise-attack-v15.1.xlsx index : 11 
current file :enterprise-attack-v15.1.xlsx index : 12 
current file :enterprise-attack-v15.1.xlsx index : 13 
current file :enterprise-attack-v15.1.xlsx index : 14 
current file :enterprise-attack-v15.1.xlsx index : 15 
current file :enterprise-attack-v15.1.xlsx index : 16 
current file :enterprise-attack-v15.1.xlsx index : 17 
current file :enterp

In [ ]:
data = pd.read_csv("outputraw.tsv", sep='\t',on_bad_lines='skip')

cleaned_data = data.dropna(subset=['Correct Answer'])


cleaned_data.to_csv("clean1.tsv", sep='\t', index=False)

print("Cleaned data saved to clean1.tsv")

Cleaned data saved to clean1.tsv


In [ ]:
ddf = pd.read_csv('clean1.tsv', delimiter='\t')

# Check for rows where Explanationere is not empty
condition = df['Explanationere'].notna() & df['Explanationere'].str.strip().astype(bool)

# Merge Option A and Option B into Option A
df.loc[condition, 'Option A'] = df['Option A'] + df['Option B']

# Shift Option B to C, Option C to D
df.loc[condition, 'Option B'] = df['Option C']
df.loc[condition, 'Option C'] = df['Option D']

# Update Correct Answer to Explanationere
df.loc[condition, 'Correct Answer'] = df['Explanationere']

# Clear the Explanationere field
df.loc[condition, 'Explanationere'] = ''

# Write the modified data to a new TSV file
df.to_csv('clean2.tsv', sep='\t', index=False)

In [ ]:
### CTI-TAA

In [1]:
def promo2 (link):
    prompt = (
            "you are a threat intel analyst given the following link that contain a attack report"
"i want you to give me a description of the attack that contains the technique IN DETAILS used and there finger print dates and techniques "
"  "
f"link : {link}"
"the output format should be a in json with the following structure {url , Text, group name} where the text is the description result in one single text block"
"IMPORTENT : do not responde with anything other then the json"

        )
    response = client.chat.completions.create(model='llama3-70b-8192', messages=[
      {
        'role': 'user',
        'content':prompt,
      },
    ])
    #print(response['message']['content'])
    return response.choices[0].message.content.strip()

In [ ]:
print(promo2("https://www.recordedfuture.com/research/china-linked-ta428-threat-group"))

Here is the output in the requested format:

```
{
"url": "https://www.recordedfuture.com/research/china-linked-ta428-threat-group",
"Text": "The China-linked TA428 threat group launched a phishing campaign targeting government and private sector organizations in the United States, Europe, and Asia. The attack begins with a spear-phishing email containing a malicious Microsoft Office document exploiting the CVE-2017-11882 vulnerability, which leads to the execution of a malicious Visual Basic for Applications (VBA) macro. The macro downloads a payload from a remote server using the Windows Management Instrumentation (WMI) command-line tool, and then injects the malware into a legitimate process using the Windows API call 'CreateProcess'. The malware, a variant of the 'Chinoxy' backdoor, establishes communication with the command and control (C2) server using Domain Generation Algorithm (DGA) and encrypts the data using RC4. The C2 servers were identified to be part of a larger network 

In [ ]:
list = [
"https://www.akamai.com/blog/security-research/2024-php-exploit-cve-one-day-after-disclosure",
"https://securelist.com/bluenoroff-new-macos-malware/111290/",
"https://www.microsoft.com/en-us/security/blog/2023/10/18/multiple-north-korean-threat-actors-exploiting-the-teamcity-cve-2023-42793-vulnerability/",
"https://www.microsoft.com/en-us/security/blog/2024/04/22/analyzing-forest-blizzards-custom-post-compromise-tool-for-exploiting-cve-2022-38028-to-obtain-credentials/",
"https://jsac.jpcert.or.jp/archive/2024/pdf/JSAC2024_1_9_takeda_furukawa_en.pdf",
"https://www.cert.ssi.gouv.fr/uploads/CERTFR-2023-CTI-009.pdf",
"https://securityintelligence.com/x-force/trickbot-conti-crypters-where-are-they-now/",
"https://malpedia.caad.fkie.fraunhofer.de/actor/fin11",
"https://blog.talosintelligence.com/talos-ir-q2-2023-quarterly-recap/",
"https://www.0ffset.net/reverse-engineering/malware-analysis/molerats-string-decryption/",
"https://www.embeeresearch.io/practical-queries-for-malware-infrastructure-part-3/",
"https://thehackernews.com/2022/02/palestinian-hackers-using-new.html",
"https://archive.is/LJFEF",
"https://go.recordedfuture.com/hubfs/reports/cta-2024-1209.pdf",
"https://jsac.jpcert.or.jp/archive/2024/pdf/JSAC2024_1_7_hara_nakajima_kawakami_en.pdf"
         ]

In [ ]:
urls = [
    # Polonium
    "https://www.microsoft.com/en-us/security/blog/2022/06/02/exposing-polonium-activity-and-infrastructure-targeting-israeli-organizations/",
    "https://www.bleepingcomputer.com/news/security/hacking-group-polonium-uses-creepy-malware-against-israel/",
    "https://www.welivesecurity.com/2022/10/11/polonium-targets-israel-creepy-malware/",

    # Longhorn
    "https://objective-see.org/blog/blog_0x68.html",
    "https://community.broadcom.com/symantecenterprise/communities/community-home/librarydocuments/viewdocument?DocumentKey=7ca2e331-2209-46a8-9e60-4cb83f9602de&CommunityKey=1ecf5f55-9545-44d6-b0f4-4e4a7f5f5e68&tab=librarydocuments",
    "https://www.secureworks.com/research/threat-profiles/platinum-terminal",

    # Lucky Cat
    "https://malgamy.github.io/malware-analysis/The-Approach-of-TA413-for-Tibetan-Targets/#third-stage",
    "https://blog.talosintelligence.com/exilerat-shares-c2-with-luckycat/",
    "https://www.broadcom.com/404-symantec?sourceURL=http://www.symantec.com/content/en/us/enterprise/media/security_response/whitepapers/the_luckycat_hackers.pdf?",

    # BlackTech
    "https://www.pwc.com/gx/en/issues/cybersecurity/cyber-threat-intelligence/cyber-year-in-retrospect/yir-cyber-threats-report-download.pdf",
    "https://blogs.jpcert.or.jp/en/2022/09/bigip-exploit.html",
    "https://jsac.jpcert.or.jp/archive/2022/pdf/JSAC2022_8_hara_en.pdf",

    # DarkHotel
    "https://web-assets.esetstatic.com/wls/2021/12/eset_jumping_the_air_gap_wp.pdf",
    "https://blog.bushidotoken.net/2020/06/deep-dive-darkhotel-apt.html",
    "https://attack.mitre.org/groups/G0012/",

    # Tick
    "https://medium.com/@Ilandu/portdoor-malware-afc9d0796cba",
    "https://cyble.com/blog/new-ransomware-groups-on-the-rise/",
    "https://www.trendmicro.com/en_us/research/21/l/collecting-in-the-dark-tropic-trooper-targets-transportation-and-government-organizations.html",

    # Turla
    "https://cyble.com/blog/tiny-backdoor-goes-undetected-suspected-turla-leveraging-msbuild-to-evade-detection/",
    "https://lab52.io/blog/pelmeni-wrapper-new-wrapper-of-kazuar-turla-backdoor/?ref=news.risky.biz",
    "https://cloud.google.com/blog/topics/threat-intelligence/turla-galaxy-opportunity/",

    # APT37
    "https://wezard4u.tistory.com/6693",
    "https://asec.ahnlab.com/en/65076/",
    "https://www.0x0v1.com/rearchive-goldbackdoor/",

    # APT3
    "https://www.googlecloudcommunity.com/gc/Community-Blog/Finding-Malware-Detecting-SOGU-with-Google-Security-Operations/ba-p/758777",
    "https://csirt-cti.net/2024/01/23/stately-taurus-targets-myanmar/",
    "https://blog.talosintelligence.com/new-zardoor-backdoor/",

    # APT17
    "https://hunt.io/blog/tracking-shadowpad-infrastructure-via-non-standard-certificates",
    "https://www.reliaquest.com/blog/anxun-and-chinese-apt-activity/",
    "https://symantec-enterprise-blogs.security.com/threat-intelligence/critical-infrastructure-attacks",

    # OilRig
    "https://www.fortinet.com/blog/threat-research/android-spynote-moves-to-crypto-currencies",
    "https://nsfocusglobal.com/apt34-unleashes-new-wave-of-phishing-attack-with-variant-of-sidetwist-trojan/",
    "https://www.bleepingcomputer.com/news/security/spynote-android-malware-infections-surge-after-source-code-leak/"
]

In [1]:
for index, value in enumerate(urls):
  res = promo2(value)
  with open("res", 'a') as file:
        file.write(res)

NameError: name 'urls' is not defined